In [8]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# prepare roi and import cloud-processing functions

In [54]:
import sys
from geeCode_qgis import exportCloudFreeSen2

exportCloudFreeSen2 is the code to process the cloudy images (a not very simple way).

## find a feature within the geojson file, just for test

In [65]:
import json

roiFile=r".\files\Multi_Class_Land_Cover_Change_AOIs.geojson"

with open(roiFile) as f:
    data = json.load(f)
    
#one feature for test
feature =data['features'][14]
#print(data['features'].dtype)
cor=feature['geometry']['coordinates']     
aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)

data['features'] is a list of dic, so the element can be selected using idxing or search (by the ['properties'] of the elements)

In [66]:
feature=next(item for item in data['features'] if item['properties']['title'] == "4666_2369_13")

idx= next((i for i, item in enumerate(data['features']) if item['properties']['title'] == "4666_2369_13"), None)

print(feature, idx)

cor=feature['geometry']['coordinates']     
aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)


{'type': 'Feature', 'properties': {'title': '4666_2369_13', 'tile_coordinates': {'x': 4666, 'y': 2369, 'z': 13}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[25.048828125, 60.19615576604439], [25.048828125, 60.21799073323445], [25.0927734375, 60.21799073323445], [25.0927734375, 60.19615576604439], [25.048828125, 60.19615576604439]]]}} 34


## buffer and visualize the aoi

In [40]:
 buffer_roi = aoi.buffer(100).bounds().simplify(ee.ErrorMargin(1, "meters"))

In [41]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [42]:
# Display the polygons by adding them to the map.
Map.centerObject(aoi,13)
Map.addLayer(aoi, {'color': 'FF0000'})
Map.addLayer(buffer_roi, {'color': 'FF0000'}, "buffer_roi")

# Compare the processed image, better?

comparing the cloud-processing approaches

In [64]:
year=2019
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

for m in [12]:#range(1,11):
    
    start_date = ee.Date.fromYMD(year, m, 1)
    end_date = ee.Date.fromYMD(year+1, 1, 1)
    
    s2 = ee.ImageCollection('COPERNICUS/S2') \
	.filterDate(start_date, end_date).select(s2_bands) \
	.filterBounds(aoi).median().clip(aoi)
    
    s2_all = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date).filterBounds(aoi)
    
    #print(s2.getInfo())

    print("the available s2 data for this roi and time: ", s2_all.size().getInfo())   
    
    s2_masked = exportCloudFreeSen2(aoi, start_date, end_date)
    s2_masked = s2_masked.select(s2_bands)
    
    #print(s2_masked.getInfo())

    Map.addLayer(s2_masked, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "s2_masked"+str(m), False)
    Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "s2"+str(m), False)
    

the available s2 data for this roi and time:  6


# when converting js files to python files

- This can be useful as there are many examples in [GEE](https://code.earthengine.google.com/) written in js.

- Some examples for converted files [here](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py)


- **from** focal_min({radius: erodePixels, kernelType: 'circle', iterations:3}) **to** focal_min(radius = erodePixels, iterations = 3)

- **from** reduceNeighborhood({reducer: ee.Reducer.max(),kernel: ee.Kernel.square(1)}) **to** reduceNeighborhood(reducer=ee.Reducer.max(), kernel=ee.Kernel.square(1))
      
- **convert tool**:

        #from geemap.conversion import *

        #js_to_python_dir(in_dir='./files/geeCode', out_dir='./files/geeCode')